### 설문조사 자료 전처리
    
    최초 작성일 : 20/03/08
    작성자 : 조예슬
    
    <작성내용>
        
            - 성별 + 연령 + 패션관심 + 안면부위 인식 선택 개수 전처리
        
        
    < 출처 >
    은주언니 햣!!▼
    https://github.com/slmteruto/CAI/blob/master/jay/Statistical%20hypothesis%20testing/Survey_preprocessing.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

from IPython.display import Image

### 대표 안면부위 인식

    성별 + 연령 + 패션관심 + 안면부위 인식 선택 개수

In [ ]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
element1 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, H")

element1.rename(columns={element1.columns[0]:"sex",
                         element1.columns[1]:"age_class",
                         element1.columns[2]:"interest"}, inplace=True)

element1["class"] = "bright"
columns = ["sex", "age_class", "interest"]


## 조화색 설문 결과 불러오기
element2 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, H")

element2.rename(columns={element2.columns[0]:"sex",
                          element2.columns[1]:"age_class",
                          element2.columns[2]:"interest"}, inplace=True)

element2["class"] = "harmony"
columns = ["sex", "age_class", "interest"]


## 두 결과 합치기
element = pd.concat([element1, element2], join='outer')


## 인덱스 조정
element.reset_index(inplace=True)
del element['index']


## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"


# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
face1 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face1.rename(columns={face1.columns[0]:"region 1",
                          face1.columns[1]:"region 2",
                          face1.columns[2]:"region 3",
                          face1.columns[3]:"region 4"}, inplace=True)

## 조화색 설문 결과 불러오기
face2 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face2.rename(columns={face2.columns[0]:"region 1",
                          face2.columns[1]:"region 2",
                          face2.columns[2]:"region 3",
                          face2.columns[3]:"region 4"}, inplace=True)


## 두 결과 합치기
face = pd.concat([face1, face2], join='outer')

## 인덱스 조정
face.reset_index(inplace=True)
del face['index']


## 중복 결과값 분리
region10, region11, region20, region21, region30, region31, region40, region41 = [], [], [], [], [], [], [], []
for i in range(int(face.shape[0])): # 210
    region10.append(face["region 1"][i].split()[0])
    region11.append(face["region 1"][i].split()[1])
    
    region20.append(face["region 2"][i].split()[0])
    region21.append(face["region 2"][i].split()[1])
    
    region30.append(face["region 3"][i].split()[0])
    region31.append(face["region 3"][i].split()[1])
    
    region40.append(face["region 4"][i].split()[0])
    region41.append(face["region 4"][i].split()[1])
    
    
## 데이터프레임 형식으로 변환
region10 = pd.DataFrame(region10, columns=["region_10"])
region11 = pd.DataFrame(region11, columns=["region_11"])
region20 = pd.DataFrame(region20, columns=["region_20"])
region21 = pd.DataFrame(region21, columns=["region_21"])
region30 = pd.DataFrame(region30, columns=["region_30"])
region31 = pd.DataFrame(region31, columns=["region_31"])
region40 = pd.DataFrame(region40, columns=["region_40"])
region41 = pd.DataFrame(region41, columns=["region_41"])


## 열 합치기
region = pd.concat([region10, region11, region20, region21, region30, region31, region40, region41], axis=1)

region.reset_index(inplace=True)
del region['index']


## 선택항목 별 부위 매칭 (①-cheek, ②-chin, ③-face, ④-eyes, ⑤-forehead)

num_list = [10, 11, 20, 21, 30, 31, 40, 41]

for j in (num_list):
    region_num = "region_" + str(j)

    for i in range(int(region.shape[0])):
        if region[region_num][i] == "①":
            region[region_num][i] = "cheek"
        elif region[region_num][i] == "②":
            region[region_num][i] = "chin"
        elif region[region_num][i] == "③":
            region[region_num][i] = "face"
        elif region[region_num][i] == "④":
            region[region_num][i] = "eyes"
        else:
            region[region_num][i] = "forehead"


SAFF = pd.concat([element, region], axis=1)

## element 부분만 가져오기

SAFF2 = SAFF.iloc[:, :4]
SAFF2

## 안면부위 선택받은 개수 세기

cheek, forehead, chin, eyes, face = [], [], [], [], []

for i in range(int(SAFF.shape[0])) :
    cheek_cnt, forehead_cnt, chin_cnt, eyes_cnt, face_cnt = [], [], [], [], []
        
    cheek_cnt = SAFF.loc[i, SAFF.loc[i] == "cheek"].count() 
    forehead_cnt = SAFF.loc[i, SAFF.loc[i] == "forehead"].count()   
    chin_cnt = SAFF.loc[i, SAFF.loc[i] == "chin"].count() 
    eyes_cnt = SAFF.loc[i, SAFF.loc[i] == "eyes"].count() 
    face_cnt = SAFF.loc[i, SAFF.loc[i] == "face"].count() 
    
    cheek.append(cheek_cnt)
    forehead.append(forehead_cnt)
    chin.append(chin_cnt)
    eyes.append(eyes_cnt)
    face.append(face_cnt)   
    
    f = pd.DataFrame([cheek, forehead, chin, eyes, face])
    face_cnt = f.T
    face_cnt.columns = ["cheek", "forehead", "chin", "eyes", "face"]
    face_cnt


SAF_Face_cnt = pd.concat([SAFF2, face_cnt], axis=1)
SAF_Face_cnt

In [ ]:
# SAF_Face_cnt 전처리 결과 저장
SAF_Face_cnt.to_csv("data/SAF_Face_cnt.csv")